# STEPs to follow
### 1. Import necessary libraries
### 2. Creating S3 bucket
### 3. Mapping train and test data in S3
### 4. Mapping path of models in S3

In [45]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import image_uris
from sagemaker.session import s3_input, Session
import pandas as pd
import numpy as np
import os

In [46]:
bucket_name = 'nba-asg-pred-2023'
my_region = boto3.session.Session().region_name #sets region of the instance
print(my_region)

us-east-1


In [47]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [48]:
# set output path where trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://nba-asg-pred-2023/xgboost-as-a-built-in-algo/output


In [64]:
#Reading in train df
try:
    model_data = pd.read_csv('./cleanedData.csv')
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


In [65]:
#train/test split
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(.7 * len(model_data))])

# Putting data into bucket

In [66]:
### Saving train dataset into bucket
pd.concat([train_data['remainder__AllStar'], train_data.drop(['remainder__AllStar'],axis=1)],
                                                                                    axis=1).to_csv('train.csv',
                                                                                                  index=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [67]:
### Saving train dataset into bucket
pd.concat([test_data['remainder__AllStar'], test_data.drop(['remainder__AllStar'],axis=1)],
                                                                                    axis=1).to_csv('test.csv',
                                                                                                  index=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


# BUILDING XGBOOST model

In [68]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
container = image_uris.retrieve(region=boto3.Session().region_name,
                         framework='xgboost', version='1.0-1')

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


In [69]:
hyperparameters = {
    'gamma':'0',
    'max_depth':'20',
    'objective':'binary:logistic',
    'num_round':'10'
}

In [70]:
#construct sagemaker estimator calling xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container,
                                         hyperparameters=hyperparameters,
                                         role=sagemaker.get_execution_role(),
                                         instance_count=1,
                                         instance_type='ml.m5.large',
                                         volume_size=5, #5GB,
                                         output_path=output_path,
                                         use_spot_instances=True,
                                         max_run=300,
                                         max_wait=600)

In [72]:
#run the estimator
estimator.fit({'train': s3_input_train, 'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2022-12-28-02-20-21-608


2022-12-28 02:20:21 Starting - Starting the training job...
2022-12-28 02:20:37 Starting - Preparing the instances for training......
2022-12-28 02:21:47 Downloading - Downloading input data...
2022-12-28 02:22:12 Training - Downloading the training image...
2022-12-28 02:22:48 Training - Training image download completed. Training in progress..[2022-12-28 02:22:53.470 ip-10-0-169-24.ec2.internal:8 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined deli

# Deploy Model

In [73]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2022-12-28-02-25-16-836
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2022-12-28-02-25-16-836
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2022-12-28-02-25-16-836


-------!

In [75]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['remainder__AllStar'], axis=1).values
xgb_predictor.serializer = csv_serializer
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')
predictions_array = np.fromstring(predictions[1:], sep=',')
print(predictions_array)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[0.02271173 0.03012614 0.03599307 ... 0.02271173 0.28918964 0.03274313]


# Delete the endpoints

In [76]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2022-12-28-02-25-16-836


[{'ResponseMetadata': {'RequestId': '504K11VN0W8TMBKC',
   'HostId': 'llWpKPBSL9iHEe4Vc6gAqhaXiXjPdVl8emNLhRZzKgvPyWwie9f1ed2MRkL5X9KXykDbcrgS83Q=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'llWpKPBSL9iHEe4Vc6gAqhaXiXjPdVl8emNLhRZzKgvPyWwie9f1ed2MRkL5X9KXykDbcrgS83Q=',
    'x-amz-request-id': '504K11VN0W8TMBKC',
    'date': 'Wed, 28 Dec 2022 02:34:42 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-12-28-02-20-21-608/profiler-output/system/incremental/2022122802/1672194120.algo-1.json'},
   {'Key': 'xgboost-as-a-built-in-algo/test/test.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-12-28-02-20-21-608/debug-output/index/000000000/000000000000_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-12-28-02-20-21-608/debug-ou